In [24]:
%load_ext autoreload
%autoreload 2

import docker
import logging
from jtldockerctl.dctl import *
import hashlib
from names_generator import generate_name


logging.basicConfig(level=logging.ERROR)
logger.setLevel(logging.ERROR)

config = {
    "images": {
        'novnc':  'ghcr.io/league-infrastructure/novnc-service:v0.1.3',
        'python_cs': 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:v0.1.5'
    },
    
    'repos': {
        'python_apprentice': 'https://github.com/league-curriculum/Python-Apprentice'
    },
    
    'repo_dir': '/mnt/student_repos',
    
    'hostname_template': "{slugify(username)}.do.jointheleague.org",
    
}

client = docker.DockerClient(base_url=f'ssh://root@64.23.135.161' )

ensure_network_exists(client, 'x11', is_external=False, network_type="bridge")
ensure_network_exists(client, 'caddy', is_external=True, network_type="bridge")
                        
image = config['images']['python_cs']
repo = config['repos']['python_apprentice']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
def make_servers(usernames):
    servers = []
    for username in usernames:
        nvc, pa = create_cs_pair(client, config, image, username, reset=True)
        yield (nvc, pa)
    return servers

print('----')
for nvc, pa in make_servers(['jed','osvaldo', 'eric']):
    print(nvc.id, pa.labels['caddy'])



----
3820892625d2be078364559d41a2da0c74e748f39f4ef2d19d651eadbdc8bf8b jed-cs.do.jointheleague.org
97aa8143ca5e4af01f8658788da848a006a250b8619f0fc62e11818353217e9a osvaldo-cs.do.jointheleague.org
1fa1fb4b1ee7d5ad0f14db168052609dd692d11771dd954830233e69f206b4a4 eric-cs.do.jointheleague.org


In [25]:

containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.labels['caddy'])
    c.remove(force=True)    

4c668f179dc25ccdd1a5b6ef42fab168c752473eebaf560023a6724643ec6df5 eric-cs.do.jointheleague.org
292c0f8c68512b8eb8d7e4ed3ab910e9ee065f27e4f97eb251ee3a7dfd83366a eric-novnc.do.jointheleague.org
08d978a27a6b303cfae95d8552bb2f39b0d0f3cf35ac00cd65b6be3f9ad39d61 osvaldo-cs.do.jointheleague.org
c7ae7219dccf9f147c8f7fc8df56e0c454ec7359494f1e995cf97cda5ae5cb33 osvaldo-novnc.do.jointheleague.org
3030cf155cc66a4ed77689d569a0e7c324f6a2e8e81240bba8cf1dbfd5d852e8 jed-cs.do.jointheleague.org
3dd29d032470b39128f1db88c0c5c655e60f9db558b89b813909b8a28f8c4ad2 jed-novnc.do.jointheleague.org
